In [4]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Reproducibility
np.random.seed(42)
random.seed(42)

# Parameters
n_rows = 21374
failure_types = [
    "oom_kill", "pod_stuck", "cpu_throttle", "node_reboot", "image_pull_error",
    "network_partition", "filesystem_error", "memory_leak", "disk_pressure",
    "pod_eviction", "high_latency", "container_crash", "init_error",
    "config_error", "dns_resolution_failure"
]
namespaces = ["default", "kube-system", "prod", "dev", "test"]
nodes = [f"node-{i:02d}" for i in range(1, 11)]

def simulate_row():
    fail = random.random() < 0.35  # ~35% failure rows
    injected = fail and (random.random() < 0.6)

    row = {
        "timestamp": (datetime(2025, 3, 1) + timedelta(seconds=random.randint(0, 30*86400))).isoformat() + "Z",
        "namespace": random.choice(namespaces),
        "pod_name": f"{random.choice(['api', 'db', 'redis'])}-deployment-{random.randint(1000, 9999)}",
        "node_name": random.choice(nodes),
        "failure_type": random.choice(failure_types) if fail else "none",
        "failure_injected": injected,
        "failure_detected": fail and random.random() < 0.7,

        # Metrics
        "cpu_usage_cores": round(np.random.normal(1.5 if fail else 0.6, 0.5), 2),
        "memory_usage_bytes": int(np.random.normal(6e9 if fail else 2e9, 1e9)),
        "disk_io_read_bytes": random.randint(1e6, 1e7),
        "disk_io_write_bytes": random.randint(1e6, 1e7),
        "network_rx_bytes": random.randint(1e5, 5e6),
        "network_tx_bytes": random.randint(1e5, 5e6),
        "oom_killed": fail and random.random() < 0.2,
        "restart_count": random.randint(5, 10) if fail else random.randint(0, 4),
        "latency_ms": round(random.uniform(200, 1000) if fail else random.uniform(5, 200), 2),
        "container_ready": not fail or random.random() > 0.4,
        "pod_scheduled": True,

        # Node metrics
        "node_cpu_allocatable_cores": random.choice([2, 4, 8]),
        "node_memory_allocatable_bytes": 16_000_000_000,

        # Label
        "will_fail_soon": fail
    }

    return row

# Generate the dataset
df = pd.DataFrame([simulate_row() for _ in range(n_rows)])

# Save
df.to_csv("podfailpred_dataset.csv", index=False)
print("Dataset saved as podfailpred_dataset.csv")


Dataset saved as podfailpred_dataset.csv
